In [1]:
# hide
# default_exp init

# Imports

In [2]:
# export

from pathlib import Path
import sys
import shutil

from fastcore.script import Param, call_parse

In [3]:
%load_ext autoreload
%autoreload 2

# Initialise a new Sciflow Project

In [4]:
# export 

env_file_blank = """export USER=
export ODBC_DRIVER=
export ODBC_USER=
export ODBC_PWD=
export ODBC_PORT=
export ODBC_HOST=
export SSL_CERTS=
export SCIFLOW_BUCKET=
export SCIFLOW_BUCKET_TABLE_ALIAS=
"""

In [5]:
# export

def write_env_file(sciflow_dir: Path = None):
    if sciflow_dir is None:
        sciflow_dir = Path('~/.sciflow').expanduser()
    env_path = Path(sciflow_dir, 'env').resolve()
    if not env_path.exists():
        if not sciflow_dir.exists():
            os.mkdir(sciflow_dir)
        with open(env_path, 'w') as env_file:
            env_file.write(env_file_blank)
        print(f'Wrote new SciFlow environment file to: {env_path}')
    else:
        print(f'Skipping SciFlow environment file creation - already exists')

In [6]:
examples_root = Path('../examples')
project_root = Path('../')
sciflow_dir = Path('test/.sciflow')
test_root = Path('test')
env_path = os.path.join(sciflow_dir, 'env')

In [7]:
if Path('test/.sciflow/env').exists():
    Path('test/.sciflow/env').unlink()

write_env_file(sciflow_dir)

with open(env_path, 'r') as env_file:
        env_file_contents = env_file.read()
assert(env_file_contents == env_file_blank)

write_env_file()

with open(env_path, 'r') as env_file:
        env_file_contents = env_file.read()
assert(env_file_contents == env_file_blank)

with open(env_path, 'w') as env_file:
    env_file.write("test")
write_env_file(sciflow_dir)
with open(env_path, 'r') as env_file:
        env_file_contents = env_file.read()
assert("test" == env_file_contents)
        
Path('test/.sciflow/env').unlink()
assert(not Path('test/.sciflow/env').exists())

write_env_file(sciflow_dir)

Wrote new SciFlow environment file to: /home/sagemaker-user/git/sciflow/nbs/test/.sciflow/env
Skipping SciFlow environment file creation - already exists
Skipping SciFlow environment file creation - already exists
Wrote new SciFlow environment file to: /home/sagemaker-user/git/sciflow/nbs/test/.sciflow/env


In [8]:
# export

def read_env_file(sciflow_dir: Path = None):
    if sciflow_dir is None:
        sciflow_dir = Path('~/.sciflow').expanduser()
    env_path = Path(sciflow_dir, 'env').resolve()
    with open(env_path, 'r') as env_file:
        lines = env_file.readlines()
    return lines

In [9]:
env_lines_wout_ppath = ["export USER='sciflow'\n",
 'export ODBC_DRIVER=/opt/odbc/xyz.so\n',
 'export SCIFLOW_BUCKET_TABLE_ALIAS=somebucket\n']

env_lines_w_1_ppath = ["export USER='sciflow'\n",
 f'export PYTHONPATH=$PYTHONPATH:{str(project_root.resolve())}\n', 
 'export SCIFLOW_BUCKET_TABLE_ALIAS=somebucket\n']

env_lines_w_2_ppath = ["export USER='sciflow'\n",
 f'export PYTHONPATH=$PYTHONPATH:{str(project_root.resolve())}:{str(examples_root.resolve())}\n', 
 'export SCIFLOW_BUCKET_TABLE_ALIAS=somebucket\n']

In [10]:
# export

def edit_pythonpath(env_lines, dir_to_add: Path):
    dir_str = str(dir_to_add.resolve())
    existing_ppath = [p for p in env_lines if p.find('PYTHONPATH') > -1]
    
    if len(existing_ppath) == 0:
        new_line = f"export PYTHONPATH=$PYTHONPATH:{dir_str}\n"
        env_lines.append(new_line)
        new_text = ''.join(env_lines)
    elif len(existing_ppath) == 1:
        prev_line = existing_ppath[0]
        if prev_line.find(dir_str) == -1:
            new_line = existing_ppath[0].replace('$PYTHONPATH:', f'$PYTHONPATH:{dir_str}')
            new_text = ''.join(env_lines).replace(prev_line, new_line)
        else:
            new_text = ''.join(env_lines)
    else:
        raise ValueError("Env file is malformed - only 1 PYTHONPATH entry should be present")
    return new_text

In [11]:
# export

def write_edited_pythonpath(project_root: Path, sciflow_dir: Path = None):
    if sciflow_dir is None:
        sciflow_dir = Path('~/.sciflow').expanduser()
    env_path = Path(sciflow_dir, 'env').resolve()
    env_lines = read_env_file(sciflow_dir)
    new_text = edit_pythonpath(env_lines, project_root)
    with open(env_path, 'w') as env_file:
            env_file.write(new_text)

In [12]:
env_vars = sorted(['USER',
 'ODBC_DRIVER',
 'ODBC_USER',
 'ODBC_PWD',
 'ODBC_PORT',
 'ODBC_HOST',
 'SSL_CERTS',
 'SCIFLOW_BUCKET',
 'SCIFLOW_BUCKET_TABLE_ALIAS',
 'PYTHONPATH'])

In [13]:
env_lines = read_env_file(sciflow_dir)
observed_vars = [x.split('=')[0].split(' ')[1] for x in edit_pythonpath(env_lines, examples_root).split('\n') if x.find('export') > -1]
assert(env_vars == sorted(observed_vars))

In [14]:
write_edited_pythonpath(test_root, sciflow_dir)

In [15]:
# export


@call_parse
def sciflow_init(project_root: Param("The root directory of the project", Path) = None, sciflow_dir: Param("The sciflow env directory", Path) = None):
    if project_root is None:
        project_root = Path('.').resolve()
    # TODO - Get latest templates files from web - if has internet connection
    
    # Create sciflow env file if it doesn't exist
    write_env_file(sciflow_dir)
    
    # Add project root to PYTHONPATH environment variable
    write_edited_pythonpath(project_root, sciflow_dir)

In [16]:
sciflow_init(examples_root, sciflow_dir)

Skipping SciFlow environment file creation - already exists
